# Universidad Autónoma de Manizales - Big Data y Visualización
## 2do Parcial
### Recuperación e integración de datos y Apache Spark

**Instrucciones:** Las preguntas $1$ a $4$ son de respuesta abierta y tienen un valor de $+0.5$. Las preguntas $5$ a $7$ son ejercicios en los cuales se califica procedimiento y tienen un valor de $+1.0$

1. Defina de forma simple y dentro del contexto del procesamiento de Big Data ¿ A qué se refiere el proceso de recuperación de información?

1. **Respuesta:** 



---
---


2. ¿Qué es un *Query language* o lenguaje de consulta? ¿Qué parámetros básicos se necesitan para una consulta? 

2. **Respuesta:** 



---
---

3. Describa y compare los modelos de datos estructurados y semi-estructurados. Mencione algunos ejemplos.

3. **Respuesta:** 



---
---

4. ¿Cuál es la principal característica de Apache Spark? Relacionada a la velocidad de procesamiento para tareas en tiempo real.

4. **Respuesta:** 



---
---

In [ ]:
# Download Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Next, we will install Apache Spark 3.0.1 with Hadoop 2.7 from here.
!wget https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
# Now, we just need to unzip that folder.
!tar xf spark-3.3.2-bin-hadoop3.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"
# Installing required packages
!pip install pyspark==3.3.2
!pip install findspark

In [ ]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as fct
sc = SparkContext()

In [ ]:
!wget -O shakespeare.txt https://www.gutenberg.org/ebooks/100.txt.utf-8

In [ ]:
import os.path
import re

def removePunctuation(text):
    text = (text
            .strip()
            .lower())
    return re.sub('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]', '', text)

fileName = os.path.join("/content/shakespeare.txt")
WordCountRDD = (sc
                .textFile(fileName, 8)
                .map(removePunctuation)
                .flatMap(lambda x: x.split())
                .filter(lambda x: x!='')
                .map(lambda x: (x,1))
                .reduceByKey(lambda x,y:x+y))

5. A partir de *WordCountRDD*:
- Muestre las 5 palabras que menos se repiten.
- Calcule el número promedio de veces que aparece una palabra.

In [ ]:
# Inserte su código aquí




---



---



In [ ]:
!wget -O sparkify-log.json https://github.com/MoraRubio/bigdata-uam/raw/main/data/sparkify-log.json

In [ ]:
ss = (SparkSession
      .builder
      .appName("parcial")
      .getOrCreate())
path = os.path.join("/content/sparkify-log.json")
user_data = ss.read.json(path)

6. A partir de *user_data*, responda: ¿cuántos usuarios de género femenino tiene la plataforma?

In [ ]:
# Inserte su código aquí




---



---



In [ ]:
!wget -q https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip
%cd ml-latest-small

In [ ]:
movies_data = ss.read.csv('/content/ml-latest-small/movies.csv', sep=',',
                          header=True, quote='"', 
                          schema='movieId INT, title STRING, genres STRING')
ratings_data = ss.read.csv('/content/ml-latest-small/ratings.csv', sep=',',
                           header=True, quote='"',
                           schema='userId INT, movieId INT, rating DOUBLE, timestamp INT')
movies_data_genre = (movies_data
                     .withColumn("genres_array", fct.split("genres", "\|"))
                     .withColumn("genre", fct.explode("genres_array"))
                     .select("movieId", "title", "genre"))
movies_data_year = (movies_data
                    .withColumn("movie year", fct.regexp_extract("title", "(\d{4})", 0)) # Generar una nueva columna con el año a partir del título
                    .select("movieId", "title", "movie year"))
movies_data_year = (movies_data_year
                    .where(movies_data_year["movie year"]>1950)
                    .where(movies_data_year["movie year"]<2023))

7. Utilizando *movies_data* y *ratings_data*, responda: ¿cuál es el género mejor valorado en cada año?

<details>
<summary>
  <font size=3 color=darkgreen><b>Recuerde el ejercicio realizado en clase</b></font>
</summary>
<p>
<b>¿Cuál es el género favorito de cada usuario?</b> Género más frecuente de las películas que el usuario valoró con 5.0.

```python
best_genre_by_user = (ratings_data
                      .join(movies_data_genre, ["movieId"], "inner")
                      .where("rating = 5.0")
                      .groupby(["userId", "genre"])
                      .count()
                      .orderBy("count", ascending=False)
                      .groupby(["userId"])
                      .agg(fct.first("genre").alias("genre"), 
                           fct.first("count").alias("count")))
best_genre_by_user.show()
```
</p>
</details>

In [ ]:
#Inserte su código aquí
